In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

In [5]:
os.chdir('/content/drive/My Drive/SunBankOCR')

In [4]:
os.chdir('HandWritingReconition/config/experiments')

In [6]:
os.chdir('resnet18_saved_model_224_224_SOTA_add801_retrain1')

In [5]:
os.listdir()

['exp_config_classifier.yaml',
 'exp_config_embedding.yaml',
 'my_classifier.yaml']

In [4]:
os.chdir('Data')

##**Download google drive 資料檔**

In [ ]:
!pip install gdown

In [ ]:
import gdown

In [ ]:
url = 'https://drive.google.com/u/0/uc?id=1DSnPaO0CYeEs4aZdOPSLdhA9JE7rae6j&export=download'
output = 'data.7z'

In [ ]:
gdown.download(url, output, False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1DSnPaO0CYeEs4aZdOPSLdhA9JE7rae6j&export=download
To: /content/drive/My Drive/SunBankOCR/Data/data.7z
625MB [00:08, 72.2MB/s]


'data.7z'

In [ ]:
os.listdir()

['data.7z', 'ESun_AI_Contest_traindata']

##**7z解壓縮**

In [ ]:
!pip install py7zr

     |████████████████████████████████| 65 kB 3.8 MB/s 
     |████████████████████████████████| 2.2 MB 14.8 MB/s 
     |████████████████████████████████| 121 kB 41.8 MB/s 
     |████████████████████████████████| 357 kB 56.9 MB/s 
     |████████████████████████████████| 1.9 MB 63.7 MB/s 


In [ ]:
from py7zr import unpack_7zarchive

In [ ]:
import shutil

In [ ]:
path = '/content/drive/My Drive/SunBankOCR/Data/'
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
shutil.unpack_archive('data.7z', path)

##**資料處理**

In [ ]:
os.chdir('ESun_AI_Contest_traindata')

In [ ]:
len(os.listdir())

801

In [ ]:
osdir = os.listdir()
for i in osdir:
  if len(os.listdir(i))==0:
    print(i)

In [ ]:
num = 0
for i in osdir:
  num = num + len(os.listdir(i))
num

68804

In [ ]:
import sys

In [ ]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/My Drive/SunBankOCR',
 '/content/drive/My Drive/SunBankOCR/HandWrittingDataset',
 '/content/drive/My Drive/SunBankOCR/HandWrittingReconition',
 '/content/drive/My Drive/SunBankOCR/HandWritingDataset',
 '/content/drive/My Drive/SunBankOCR/HandWritingReconition']

In [ ]:
sys.path.append('/content/drive/My Drive/SunBankOCR')

In [ ]:
sys.path.append('/content/drive/My Drive/SunBankOCR/HandWritingDataset')

In [ ]:
sys.path.append('/content/drive/My Drive/SunBankOCR/HandWritingReconition')

In [ ]:
os.listdir()

['split_dataset.py',
 'pipeline.py',
 '__init__.py',
 'data_preprocess.py',
 '__pycache__']

##**SplitDataset**

In [ ]:
import os
import random


import shutil


class SplitDataset():
    def __init__(self, dataset_dir, saved_dataset_dir, train_ratio=0.8, test_ratio=0.15, show_progress=False):
        self.dataset_dir = dataset_dir
        self.saved_dataset_dir = saved_dataset_dir
        self.saved_train_dir = saved_dataset_dir + "/train/"
        self.saved_valid_dir = saved_dataset_dir + "/valid/"
        self.saved_test_dir = saved_dataset_dir + "/test/"


        self.train_ratio = train_ratio
        self.test_radio = test_ratio
        self.valid_ratio = 1 - train_ratio - test_ratio

        self.train_file_path = []
        self.valid_file_path = []
        self.test_file_path = []

        self.index_label_dict = {}

        self.show_progress = show_progress

        if not os.path.exists(self.saved_train_dir):
            os.mkdir(self.saved_train_dir)
        if not os.path.exists(self.saved_test_dir):
            os.mkdir(self.saved_test_dir)
        if not os.path.exists(self.saved_valid_dir):
            os.mkdir(self.saved_valid_dir)


    def __get_label_names(self):
        label_names = []
        for item in os.listdir(self.dataset_dir):
            item_path = os.path.join(self.dataset_dir, item)
            if os.path.isdir(item_path):
                label_names.append(item)
        return label_names

    def __get_all_file_path(self):
        all_file_path = []
        index = 0
        for file_type in self.__get_label_names():
            self.index_label_dict[index] = file_type
            index += 1
            type_file_path = os.path.join(self.dataset_dir, file_type)
            file_path = []
            for file in os.listdir(type_file_path):
                single_file_path = os.path.join(type_file_path, file)
                file_path.append(single_file_path)
            all_file_path.append(file_path)
        return all_file_path

    def __copy_files(self, type_path, type_saved_dir):
        for item in type_path:
            src_path_list = item[1]
            dst_path = type_saved_dir + "%s/" % (item[0])
            if not os.path.exists(dst_path):
                os.mkdir(dst_path)
            for src_path in src_path_list:
                shutil.copy(src_path, dst_path)
                if self.show_progress:
                    print("Copying file "+src_path+" to "+dst_path)

    def __split_dataset(self):
        all_file_paths = self.__get_all_file_path()
        for index in range(len(all_file_paths)):
            file_path_list = all_file_paths[index]
            file_path_list_length = len(file_path_list)
            random.shuffle(file_path_list)

            train_num = int(file_path_list_length * self.train_ratio)
            test_num = int(file_path_list_length * self.test_radio)

            self.train_file_path.append([self.index_label_dict[index], file_path_list[: train_num]])
            self.test_file_path.append([self.index_label_dict[index], file_path_list[train_num:train_num + test_num]])
            self.valid_file_path.append([self.index_label_dict[index], file_path_list[train_num + test_num:]])

    def start_splitting(self):
        self.__split_dataset()
        self.__copy_files(type_path=self.train_file_path, type_saved_dir=self.saved_train_dir)
        self.__copy_files(type_path=self.valid_file_path, type_saved_dir=self.saved_valid_dir)
        self.__copy_files(type_path=self.test_file_path, type_saved_dir=self.saved_test_dir)

In [ ]:
def pipeline():
  dataset_dir = '/content/drive/My Drive/SunBankOCR/Data/ESun_AI_Contest_traindata'
  saved_dataset_dir = '/content/drive/My Drive/SunBankOCR/Data'

  split_dataset = SplitDataset(dataset_dir=dataset_dir,
                          saved_dataset_dir=saved_dataset_dir,
                          train_ratio=0.8, 
                          test_ratio=0.15,
                          show_progress=True)
  split_dataset.start_splitting()

In [ ]:
pipeline()

串流輸出內容已截斷至最後 5000 行。
Copying file /content/drive/My Drive/SunBankOCR/Data/ESun_AI_Contest_traindata/800/45754_川.jpg to /content/drive/My Drive/SunBankOCR/Data/test/800/
Copying file /content/drive/My Drive/SunBankOCR/Data/ESun_AI_Contest_traindata/800/18666_辰.jpg to /content/drive/My Drive/SunBankOCR/Data/test/800/
Copying file /content/drive/My Drive/SunBankOCR/Data/ESun_AI_Contest_traindata/800/65408_常.jpg to /content/drive/My Drive/SunBankOCR/Data/test/800/
Copying file /content/drive/My Drive/SunBankOCR/Data/ESun_AI_Contest_traindata/800/23935_琇.jpg to /content/drive/My Drive/SunBankOCR/Data/test/800/
Copying file /content/drive/My Drive/SunBankOCR/Data/ESun_AI_Contest_traindata/800/9290_風.jpg to /content/drive/My Drive/SunBankOCR/Data/test/800/
Copying file /content/drive/My Drive/SunBankOCR/Data/ESun_AI_Contest_traindata/800/2768_珊.jpg to /content/drive/My Drive/SunBankOCR/Data/test/800/
Copying file /content/drive/My Drive/SunBankOCR/Data/ESun_AI_Contest_traindata/800/11175_五.jp

##**config**

In [19]:
!pip install yacs

In [20]:
from yacs.config import CfgNode as ConfigurationNode

# YACS overwrite these settings using YAML
__C = ConfigurationNode()

# DATASET
__C.DATASET = ConfigurationNode()
__C.DATASET.SOURCE = "Inner"
__C.DATASET.DIR = str()
__C.DATASET.TRAIN_DIR = str()
__C.DATASET.VALID_DIR = str()
__C.DATASET.TEST_DIR =  str()
__C.DATASET.IMAGE_HEIGHT = int()
__C.DATASET.IMAGE_WIDTH = int()
__C.DATASET.CHANNELS = 3

# MODEL
__C.MODEL = ConfigurationNode()
__C.MODEL.BACKBONE    = "resnet18" #resnet34 resnet50 resnet101 resnet152
__C.MODEL.NUM_CLASSES = 801
__C.MODEL.BACKBONE_OUTPUT_SIZE = 512
__C.MODEL.EMBEDDING_SIZE = int()

# TRAINING
__C.TRAINING = ConfigurationNode()
__C.TRAINING.EPOCH             = 10
__C.TRAINING.BATCH_SIZE        = 1
__C.TRAINING.LEARNING_RATE     = 0.001
__C.TRAINING.SAVE_MODEL_DIR    = str()
__C.TRAINING.LOSS              = "balance_loss"
__C.TRAINING.SAVE_TIMES        = 1  #cannot work
__C.TRAINING.RETRAIN_MODEL_DIR = str()


# TESTING
__C.TESTING = ConfigurationNode()
__C.TESTING.BATCH_SIZE         = 100

def get_cfg_defaults():
    return __C.clone()

In [21]:
image_height = None
image_width = None
channels = None

##**models**

In [22]:
import tensorflow as tf

class Classifier(tf.keras.layers.Layer):
    def __init__(self, class_num):
        super().__init__()
        self.fc     = tf.keras.layers.Dense(units=class_num, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, **kwargs):
        outputs = self.fc(inputs)
        return outputs


class BasicBlock(tf.keras.layers.Layer):

    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        if stride != 1:
            self.downsample = tf.keras.Sequential()
            self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num,
                                                       kernel_size=(1, 1),
                                                       strides=stride))
            self.downsample.add(tf.keras.layers.BatchNormalization())
        else:
            self.downsample = lambda x: x

    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output


class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, filter_num, stride=1):
        super(BottleNeck, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(filters=filter_num * 4,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        self.bn3 = tf.keras.layers.BatchNormalization()

        self.downsample = tf.keras.Sequential()
        self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num * 4,
                                                   kernel_size=(1, 1),
                                                   strides=stride))
        self.downsample.add(tf.keras.layers.BatchNormalization())

    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv3(x)
        x = self.bn3(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output


def make_basic_block_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BasicBlock(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BasicBlock(filter_num, stride=1))

    return res_block


def make_bottleneck_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BottleNeck(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BottleNeck(filter_num, stride=1))

    return res_block

In [23]:
class ResNetTypeI(tf.keras.Model):
    def __init__(self, layer_params, config):
        super(ResNetTypeI, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=2,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                               strides=2,
                                               padding="same")

        self.layer1 = make_basic_block_layer(filter_num=64,
                                             blocks=layer_params[0])
        self.layer2 = make_basic_block_layer(filter_num=128,
                                             blocks=layer_params[1],
                                             stride=2)
        self.layer3 = make_basic_block_layer(filter_num=256,
                                             blocks=layer_params[2],
                                             stride=2)
        self.layer4 = make_basic_block_layer(filter_num=512,
                                             blocks=layer_params[3],
                                             stride=2)

        #self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        # self.fc1     = tf.keras.layers.Dense(units=config.MODEL.BACKBONE_OUTPUT_SIZE, activation=tf.keras.activations.relu)
        # self.fc1     = tf.keras.layers.Dense(units=config.MODEL.NUM_CLASSES, activation=tf.keras.activations.relu)
        # self.fc2     = tf.keras.layers.Dense(units=config.MODEL.NUM_CLASSES, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        print("look shape", x.shape)
        output = self.avgpool(x)
        print("look shape", output.shape)
        # output = self.fc1(x)
        print("look shape", output.shape)
        # output = self.fc2(output)


        return output


class ResNetTypeII(tf.keras.Model):
    def __init__(self, layer_params, config):
        super(ResNetTypeII, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=2,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                               strides=2,
                                               padding="same")

        self.layer1 = make_bottleneck_layer(filter_num=64,
                                            blocks=layer_params[0])
        self.layer2 = make_bottleneck_layer(filter_num=128,
                                            blocks=layer_params[1],
                                            stride=2)
        self.layer3 = make_bottleneck_layer(filter_num=256,
                                            blocks=layer_params[2],
                                            stride=2)
        self.layer4 = make_bottleneck_layer(filter_num=512,
                                            blocks=layer_params[3],
                                            stride=2)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        # self.fc1     = tf.keras.layers.Dense(units=config.MODEL.BACKBONE_OUTPUT_SIZE, activation=tf.keras.activations.relu)
        # self.fc2     = tf.keras.layers.Dense(units=config.MODEL.NUM_CLASSES, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        output = self.avgpool(x)
        # output = self.fc1(output)
        # output = self.fc2(output)
        return output


def resnet_18(config):
    return ResNetTypeI(layer_params=[2, 2, 2, 2], config=config)


def resnet_34(config):
    return ResNetTypeI(layer_params=[3, 4, 6, 3], config=config)


def resnet_50(config):
    return ResNetTypeII(layer_params=[3, 4, 6, 3], config=config)


def resnet_101(config):
    return ResNetTypeII(layer_params=[3, 4, 23, 3], config=config)

##**Loss**

In [24]:
from enum import Enum

class LossController:
    class Type(Enum):
        sparse_category_loss = "sparse_category_loss"
        balance_loss = "balance_loss"
    
    def __init__(self, loss_fn, alpha=None, beta=None, class_num=None):
        try:
            loss = self.Type(loss_fn)
        except ValueError:
            raise Exception(f"your input is {loss_fn}, please input {[Type.value for Type in self.Type]}")

        if loss == self.Type.sparse_category_loss:
            self.loss = tf.keras.losses.SparseCategoricalCrossentropy()
            print("SparseCategoricalCrossentropy")
        elif loss == self.Type.balance_loss:
            self.loss = BalanceLoss(alpha,beta,class_num)
            print("balance_loss")

    def get(self):
        return self.loss

##**train**

In [12]:
import tensorflow as tf
import pathlib
import random
#from ..external import image_config
import os

def load_and_preprocess_image(img_path):
    global image_height, image_width, channels
    print("image_height",image_height)
    # read pictures
    img_raw = tf.io.read_file(img_path)
    # decode pictures
    img_tensor = tf.image.decode_jpeg(img_raw, channels=channels)
    # resize
    img_tensor = tf.image.resize(img_tensor, [image_height, image_width])
    img_tensor = tf.cast(img_tensor, tf.float32)
    # normalization
    img = img_tensor / 255.0
    return img

def get_images_and_labels(data_root_dir):
    # get all images' paths (format: string)
    print("data_root_dir",data_root_dir)
    data_root = pathlib.Path(data_root_dir)
    all_image_path = [str(path) for path in list(data_root.glob('*/*'))]
    # get labels' names
    label_names = sorted(item.name for item in data_root.glob('*/'))
    if '.DS_Store' in label_names:
        label_names.remove('.DS_Store')
    # dict: {label : index}
    label_to_index = dict((label, index) for index, label in enumerate(label_names))
    # get all images' labels
    all_image_label = [label_to_index[pathlib.Path(single_image_path).parent.name] for single_image_path in all_image_path]
    # print("look_here",label_to_index)
    # print("look", all_image_path[:10], all_image_label[:10])
    return all_image_path, all_image_label


def get_dataset(dataset_root_dir):
    all_image_path, all_image_label = get_images_and_labels(data_root_dir=dataset_root_dir)
    # print("image_path: {}".format(all_image_path[:]))
    # print("image_label: {}".format(all_image_label[:]))
    
    print("before",all_image_path[:3])
    # shuffle data
    combine = list(zip(all_image_path,all_image_label))
    random.shuffle(combine)
    all_image_path, all_image_label = zip(*combine)
    all_image_path = list(all_image_path)
    all_image_label = list(all_image_label)
    print("after",all_image_path[:3])

    # load the dataset and preprocess images
    image_dataset = tf.data.Dataset.from_tensor_slices(all_image_path).map(load_and_preprocess_image)
    label_dataset = tf.data.Dataset.from_tensor_slices(all_image_label)
    dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
    image_count = len(all_image_path)
    print("image_count",image_count)
    print(all_image_path[0])
    return dataset, image_count


def generate_datasets(config):
    print("second")
    print("config.DATASET.TRAIN_DIR",config.DATASET.TRAIN_DIR)
    train_dataset, train_count = get_dataset(dataset_root_dir=config.DATASET.TRAIN_DIR)
    valid_dataset, valid_count = get_dataset(dataset_root_dir=config.DATASET.VALID_DIR)
    test_dataset, test_count = get_dataset(dataset_root_dir=config.DATASET.TEST_DIR)


    # read the original_dataset in the form of batch
    # train_dataset = train_dataset.shuffle(buffer_size=train_count).batch(batch_size=config.TRAINING.BATCH_SIZE)
    train_dataset = train_dataset.batch(batch_size=config.TRAINING.BATCH_SIZE)
    valid_dataset = valid_dataset.batch(batch_size=config.TRAINING.BATCH_SIZE)
    test_dataset = test_dataset.batch(batch_size=config.TRAINING.BATCH_SIZE) #要改testing

    return train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count

In [56]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import math
from tensorflow.keras.models import Model


class CategoryModel(tf.keras.Model):
    
    def __init__(self, backbone, classifier):
        super().__init__()
        self.backbone = backbone
        self.classifier = classifier

    def call(self, inputs, training=None, **kwargs):
        x_in = tf.keras.layers.Input(shape=(224, 224, 3),name="x_in")
        x = x_in
        output = self.backbone(x)
        #x = self.backbone(inputs)
        #output = self.classifier(x)
        #mymodel = Model(inputs=x_in, outputs=output)
        return Model(inputs=x_in, outputs=output)

#已經改成兩層輸出層
class ClassifierTrainer:

    def __init__(self, config=None, alpha=None, beta=None, retrain=False):
        self.config = config
        self.alpha = alpha
        self.beta  = beta
        self.retrain = retrain

    def get_model(self):
        classifier = Classifier(self.config.MODEL.NUM_CLASSES)
        backbone = resnet_50(self.config)
        if self.config.MODEL.BACKBONE == "resnet18":
            backbone = resnet_18(self.config)
        if self.config.MODEL.BACKBONE == "resnet34":
            backbone = resnet_34(self.config)
        if self.config.MODEL.BACKBONE == "resnet101":
            backbone = resnet_101(self.config)
        if self.config.MODEL.BACKBONE == "resnet152":
            backbone = resnet_152(self.config)
        #x_in = tf.keras.layers.Input(shape=(None, self.config.DATASET.IMAGE_HEIGHT, self.config.DATASET.IMAGE_WIDTH, self.config.DATASET.CHANNELS),name="x_in")
        #category_model = CategoryModel(backbone=backbone, classifier=classifier)
        #category_model = CategoryModel(backbone=backbone, classifier=classifier)

        #category_model.build(input_shape=(None, self.config.DATASET.IMAGE_HEIGHT, self.config.DATASET.IMAGE_WIDTH, self.config.DATASET.CHANNELS))
        #x_in = tf.keras.layers.Input(shape=(224, 224, 3),name="x_in")
        #category_model = Model(inputs=x_in, outputs=category_model)

        globalAP = tf.keras.layers.GlobalAveragePooling2D(name="GlobalAP")
        
        x_in = tf.keras.layers.Input(shape=(224, 224, 3),name="x_in")
        x = x_in
        x = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=2,
                                            padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.MaxPool2D(pool_size=(3, 3),strides=2, padding="same")(x)

        x = make_basic_block_layer(filter_num=64, blocks=2)(x)
        x = make_basic_block_layer(filter_num=128, blocks=2, stride=2)(x)
        x = make_basic_block_layer(filter_num=256, blocks=2, stride=2)(x)
        x = make_basic_block_layer(filter_num=512, blocks=2, stride=2)(x)

        model = Model(x_in, x)
        x = x_in
        x = model(x)
        x = globalAP(x)
        x = tf.keras.layers.Dense(units=801, activation=tf.keras.activations.softmax)(x)
        category_model1 = Model(inputs=x_in, outputs=x)

        category_model1.summary()
        return category_model1

    @tf.function
    def train_step(self, images, labels, model, loss_object, optimizer, train_loss, train_accuracy):
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            loss = loss_object(y_true=labels, y_pred=predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)

    @tf.function
    def valid_step(self, images, labels, model, loss_object, valid_loss, valid_accuracy):
        predictions = model(images, training=False)
        v_loss = loss_object(labels, predictions)

        valid_loss(v_loss)
        valid_accuracy(labels, predictions)

    def train(self):
        # GPU settings
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
                print("=====")
                print("get_gpu")
                print("=====")
            # get the original_dataset
        
        train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count = generate_datasets(self.config)
         # create model
        model = self.get_model()
        if self.retrain:
            model.load_weights(filepath=self.config.TRAINING.RETRAIN_MODEL_DIR)


        # define loss and optimizer
        loss_controller = LossController(loss_fn=self.config.TRAINING.LOSS, alpha=self.alpha, beta=self.beta, class_num=self.config.MODEL.NUM_CLASSES)
        loss_object     = loss_controller.get()
        # loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
        optimizer = tf.keras.optimizers.Adam(lr=self.config.TRAINING.LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        # optimizer = tf.keras.optimizers.Adadelta()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        valid_loss = tf.keras.metrics.Mean(name='valid_loss')
        valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

        # start training
        for epoch in range(self.config.TRAINING.EPOCH):
            train_loss.reset_states()
            train_accuracy.reset_states()
            valid_loss.reset_states()
            valid_accuracy.reset_states()
            step = 0
            for images, labels in train_dataset:
                # print(labels)
                step += 1
                self.train_step(images, labels, model, loss_object, optimizer, train_loss, train_accuracy)
                print("Epoch: {}/{}, step: {}/{}, loss: {:.5f}, accuracy: {:.5f}".format(epoch + 1,
                                                                                        self.config.TRAINING.EPOCH,
                                                                                        step,
                                                                                        math.ceil(train_count / self.config.TRAINING.BATCH_SIZE),
                                                                                        train_loss.result(),
                                                                                        train_accuracy.result()))

            
            if (epoch+1)%math.ceil(self.config.TRAINING.EPOCH/self.config.TRAINING.SAVE_TIMES)==0:
                for valid_images, valid_labels in valid_dataset:
                    self.valid_step(valid_images, valid_labels, model, loss_object, valid_loss, valid_accuracy)

            print("Epoch: {}/{}, train loss: {:.5f}, train accuracy: {:.5f}, "
                "valid loss: {:.5f}, valid accuracy: {:.5f}".format(epoch + 1,
                                                                    self.config.TRAINING.EPOCH,
                                                                    train_loss.result(),
                                                                    train_accuracy.result(),
                                                                    valid_loss.result(),
                                                                    valid_accuracy.result()), file=open(f"retrain{self.config.MODEL.BACKBONE}_{self.config.TRAINING.LOSS}_{self.config.TRAINING.EPOCH}_train_refine_all_data_patch_data.txt", "a+"))

            # if (epoch+1)%math.ceil(self.config.TRAINING.EPOCH/self.config.TRAINING.SAVE_TIMES) == 0:
            #     temp = self.config.TRAINING.SAVE_MODEL_DIR
            #     temp_file = temp+str(epoch+1)+"epoch"
            #     print(temp_file)
            #     model.save_weights(filepath=temp_file, save_format='tf')

        model.save_weights(filepath=self.config.TRAINING.SAVE_MODEL_DIR, save_format='tf')

##**evaluate**

In [25]:
def test_step(images, labels, model, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    argmax_predictions = tf.argmax(predictions, axis=1)
    print(argmax_predictions, labels, t_loss)
    test_loss(t_loss)
    test_accuracy(labels, predictions)
    return labels, argmax_predictions

def evaluation(config):
    # GPU settings
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    print("開始進行測試~~~~~")
    # get the original_dataset
    train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count = generate_datasets(config)
    print("test",test_dataset, test_count)
    # load the model
    trainer = ClassifierTrainer(config)
    model = trainer.get_model()
    model.load_weights(filepath=config.TRAINING.SAVE_MODEL_DIR)
    weights_list = model.get_weights()
    # print(weights_list[-2].shape,weights_list[-3].shape,weights_list[-1])
    
    # Get the accuracy on the test set
    loss_object = tf.keras.metrics.SparseCategoricalCrossentropy()
    test_loss = tf.keras.metrics.Mean()
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    collect_predict, collect_label = [],[]
    count = 0
    for test_images, test_labels in test_dataset:
        labels, argmax_predictions = test_step(test_images, test_labels, model, loss_object, test_loss, test_accuracy)
        # loss應該是一直update 會把數據一直疊加
        print("loss: {:.5f}, test accuracy: {:.5f}".format(test_loss.result(),
                                                           test_accuracy.result()))
        collect_label.extend(labels)
        collect_predict.extend(argmax_predictions)
    print("The accuracy on test set is: {:.3f}%".format(test_accuracy.result()*100))
    
    con_mat = tf.math.confusion_matrix(labels=collect_label, predictions=collect_predict).numpy()
    print(con_mat,con_mat.shape)
    print(len(collect_predict), collect_predict[-30:],collect_predict.count(800), collect_label.count(800))
    from collections import Counter
    collect_label = [i.numpy() for i in collect_label]
    collect_predict = [i.numpy() for i in collect_predict]
    print(Counter(collect_label))
    print(Counter(collect_predict))
    return con_mat

In [26]:
def load_and_preprocess_image(img_path):
    global image_height, image_width, channels
    print("image_height",image_height)
    # read pictures
    img_raw = tf.io.read_file(img_path)
    # decode pictures
    img_tensor = tf.image.decode_jpeg(img_raw, channels=channels)
    # resize
    img_tensor = tf.image.resize(img_tensor, [image_height, image_width])
    img_tensor = tf.cast(img_tensor, tf.float32)
    # normalization
    img = img_tensor / 255.0
    return img




def test_feature_map(image, config):
    if len(image.shape) == 3:
        image = tf.expand_dims(image, 0)
    trainer = ClassifierTrainer(config)
    model = trainer.get_model()
    model.load_weights(filepath=config.TRAINING.RETRAIN_MODEL_DIR)
    predictions = model(image, training=False)
    print(predictions)
    print(model.layers)
    print(model.layers[0].output, model.layers[0].output.shape)
    outputs = [layer.output for layer in model.layers] 
    print("outputs",outputs)

def test_step(images, labels, model, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)
    #model.get
    t_loss = loss_object(labels, predictions)
    argmax_predictions = tf.argmax(predictions, axis=1)
    print(argmax_predictions, labels, t_loss)
    test_loss(t_loss)
    test_accuracy(labels, predictions)
    return labels, argmax_predictions

def evaluation(config):
    # GPU settings
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    print("開始進行測試~~~~~")
    # get the original_dataset
    train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count = generate_datasets(config)
    print("test",test_dataset, test_count)
    # load the model
    trainer = ClassifierTrainer(config)
    model = trainer.get_model()
    model.load_weights(filepath=config.TRAINING.SAVE_MODEL_DIR)
    weights_list = model.get_weights()
    # print(weights_list[-2].shape,weights_list[-3].shape,weights_list[-1])
    
    # Get the accuracy on the test set
    loss_object = tf.keras.metrics.SparseCategoricalCrossentropy()
    test_loss = tf.keras.metrics.Mean()
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    collect_predict, collect_label = [],[]
    count = 0
    for test_images, test_labels in test_dataset:
        labels, argmax_predictions = test_step(test_images, test_labels, model, loss_object, test_loss, test_accuracy)
        # loss應該是一直update 會把數據一直疊加
        print("loss: {:.5f}, test accuracy: {:.5f}".format(test_loss.result(),
                                                           test_accuracy.result()))
        collect_label.extend(labels)
        collect_predict.extend(argmax_predictions)
    print("The accuracy on test set is: {:.3f}%".format(test_accuracy.result()*100))
    
    con_mat = tf.math.confusion_matrix(labels=collect_label, predictions=collect_predict).numpy()
    print(con_mat,con_mat.shape)
    print(len(collect_predict), collect_predict[-30:],collect_predict.count(800), collect_label.count(800))
    from collections import Counter
    collect_label = [i.numpy() for i in collect_label]
    collect_predict = [i.numpy() for i in collect_predict]
    print(Counter(collect_label))
    print(Counter(collect_predict))
    return con_mat

##**sample.py**

In [29]:
#from HandWritingReconition.config import get_cfg_defaults
#from HandWritingReconition.train.classifier_trainer import ClassifierTrainer
#from HandWritingReconition.train.embedding_trainer import EmbeddingTrainer
#from HandWritingReconition.external import image_config 
#from HandWritingReconition.evaluate import evaluation
import os

def load_config(config_file):
    cfg = get_cfg_defaults()
    cfg.merge_from_file(config_file)
    cfg.freeze()
    return cfg

def classifier_train(cfg, alpha, beta, retrain):
    trainer = ClassifierTrainer(config=cfg, alpha=alpha, beta=beta, retrain=retrain)
    # trainer = TwoFFNNTrainer(config=cfg)
    trainer.train()

def develop_embedding_train(cfg):
    trainer = EmbeddingTrainer(config=cfg)
    trainer.train()

def check_data_positive_negative_rate(cfg):
    train_path = cfg.DATASET.TRAIN_DIR
    collect_data_size= []
    for i in range(cfg.MODEL.NUM_CLASSES):
        path = os.path.join(train_path,str(i))
        collect_data_size.append(len(os.listdir(path)))
    print(collect_data_size)
    total_num = sum(collect_data_size)
    alpha = [(total_num-class_num)/total_num for class_num in collect_data_size]
    beta = [(class_num)/total_num for class_num in collect_data_size]
    # print(alpha,beta)
    return alpha,beta

In [28]:
import os
config_file=f"HandWritingReconition{os.sep}config{os.sep}experiments{os.sep}my_classifier.yaml"
cfg = load_config(config_file)
# 模型訓練
image_height = cfg.DATASET.IMAGE_HEIGHT
image_width = cfg.DATASET.IMAGE_WIDTH
channels = cfg.DATASET.CHANNELS
print(cfg)
alpha,beta = check_data_positive_negative_rate(cfg)
classifier_train(cfg, alpha, beta, retrain=False)
# develop_embedding_train(cfg)    

    
    # from external import config
    # config.save_model_dir = "saved_model/resnet18_saved_model/model"
    # config.dataset_dir = "dataset/"
    # from evaluate import evaluation
    # evaluation(cfg)

NameError: ignored

In [31]:
config_file=f"HandWritingReconition{os.sep}config{os.sep}experiments{os.sep}my_classifier.yaml"
cfg = load_config(config_file)
# 模型訓練
image_height = cfg.DATASET.IMAGE_HEIGHT
image_width = cfg.DATASET.IMAGE_WIDTH
channels = cfg.DATASET.CHANNELS
print(cfg)

DATASET:
  CHANNELS: 3
  DIR: dataset
  IMAGE_HEIGHT: 224
  IMAGE_WIDTH: 224
  SOURCE: Inner
  TEST_DIR: /content/drive/My Drive/SunBankOCR/Data/test
  TRAIN_DIR: /content/drive/My Drive/SunBankOCR/Data/train
  VALID_DIR: /content/drive/My Drive/SunBankOCR/Data/valid
MODEL:
  BACKBONE: resnet18
  BACKBONE_OUTPUT_SIZE: 512
  EMBEDDING_SIZE: 0
  NUM_CLASSES: 801
TESTING:
  BATCH_SIZE: 100
TRAINING:
  BATCH_SIZE: 32
  EPOCH: 2
  LEARNING_RATE: 0.0005
  LOSS: sparse_category_loss
  RETRAIN_MODEL_DIR: HandWritingReconition/saved_model/resnet18_saved_model_224_224_SOTA/model
  SAVE_MODEL_DIR: HandWritingReconition/saved_model/resnet18_saved_model_224_224_SOTA_add801_retrain1/model
  SAVE_TIMES: 2


In [18]:
alpha,beta = check_data_positive_negative_rate(cfg)
#評估
con_mat = evaluation(cfg)
import json
with open("result_224_224_train_valid_test_8_0_2_0relabel_resnet50.json","w") as fp:
    con_mat = con_mat.tolist()
    json.dump(con_mat,fp)

DATASET:
  CHANNELS: 3
  DIR: dataset
  IMAGE_HEIGHT: 224
  IMAGE_WIDTH: 224
  SOURCE: Inner
  TEST_DIR: /content/drive/My Drive/SunBankOCR/Data/test
  TRAIN_DIR: /content/drive/My Drive/SunBankOCR/Data/train
  VALID_DIR: /content/drive/My Drive/SunBankOCR/Data/valid
MODEL:
  BACKBONE: resnet18
  BACKBONE_OUTPUT_SIZE: 512
  EMBEDDING_SIZE: 0
  NUM_CLASSES: 801
TESTING:
  BATCH_SIZE: 100
TRAINING:
  BATCH_SIZE: 32
  EPOCH: 2
  LEARNING_RATE: 0.0005
  LOSS: sparse_category_loss
  RETRAIN_MODEL_DIR: HandWritingReconition/saved_model/resnet18_saved_model_224_224_SOTA/model
  SAVE_MODEL_DIR: HandWritingReconition/saved_model/resnet18_saved_model_224_224_SOTA_add801_retrain1/model
  SAVE_TIMES: 2
[52, 64, 53, 57, 50, 62, 68, 68, 48, 61, 57, 51, 60, 48, 43, 42, 50, 55, 48, 50, 52, 48, 59, 50, 51, 56, 11, 50, 51, 63, 30, 59, 23, 56, 28, 54, 51, 57, 55, 36, 43, 58, 54, 64, 60, 53, 44, 57, 57, 61, 54, 60, 56, 63, 59, 46, 44, 44, 35, 58, 59, 67, 62, 60, 52, 57, 31, 60, 47, 59, 65, 52, 61, 63, 56,

AttributeError: ignored

In [57]:
trainer = ClassifierTrainer(cfg)
model = trainer.get_model()
model.load_weights(filepath=cfg.TRAINING.SAVE_MODEL_DIR)
weights_list = model.get_weights()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x_in (InputLayer)            [(None, 224, 224, 3)]     0         
_________________________________________________________________
model_3 (Functional)         (None, 7, 7, 512)         11190912  
_________________________________________________________________
GlobalAP (GlobalAveragePooli (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 801)               410913    
Total params: 11,601,825
Trainable params: 11,592,225
Non-trainable params: 9,600
_________________________________________________________________


AssertionError: ignored

In [24]:
import numpy as np

In [27]:
weights_list[0].shape

(7, 7, 3, 64)

In [28]:
from tensorflow.keras.models import Model

In [38]:
_, _, test_dataset, _, _, test_count = generate_datasets(cfg)

second
config.DATASET.TRAIN_DIR /content/drive/My Drive/SunBankOCR/Data/train
data_root_dir /content/drive/My Drive/SunBankOCR/Data/train
before ['/content/drive/My Drive/SunBankOCR/Data/train/0/11016_宋.jpg', '/content/drive/My Drive/SunBankOCR/Data/train/0/67970_宋.jpg', '/content/drive/My Drive/SunBankOCR/Data/train/0/32433_宋.jpg']
after ['/content/drive/My Drive/SunBankOCR/Data/train/553/63559_應.jpg', '/content/drive/My Drive/SunBankOCR/Data/train/345/59828_琴.jpg', '/content/drive/My Drive/SunBankOCR/Data/train/800/19513_加.jpg']
image_height 224
image_count 54712
/content/drive/My Drive/SunBankOCR/Data/train/553/63559_應.jpg
data_root_dir /content/drive/My Drive/SunBankOCR/Data/valid
before ['/content/drive/My Drive/SunBankOCR/Data/valid/0/4555_宋.jpg', '/content/drive/My Drive/SunBankOCR/Data/valid/0/17534_宋.jpg', '/content/drive/My Drive/SunBankOCR/Data/valid/0/60432_宋.jpg']
after ['/content/drive/My Drive/SunBankOCR/Data/valid/509/6076_技.jpg', '/content/drive/My Drive/SunBankOCR/Dat

In [40]:
test_dataset[0][0].shape

TypeError: ignored

In [37]:
predictions = model(img[0], training=False)
print(predictions)
print(model.layers)
print(model.layers[0].output, model.layers[0].output.shape)
outputs = [layer.output for layer in model.layers] 
print("outputs",outputs)

NameError: ignored

In [58]:
model_1 = Model(inputs=tf.keras.Input(shape=(224, 224, 3)), outputs=model.layers[0].output)

ValueError: ignored

In [39]:
model.layers[0].conv1.output

AttributeError: ignored

In [35]:
dir(model.layers[0])

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_cast_single_input',
 '_check_call_args',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_cluster_coordinator',
 '_compile_wa

In [ ]:
model = ResNet18()